### Commands

In [16]:
from firecloud import api as firecloud_api
import fc_interface

In [17]:
%load_ext autoreload
%autoreload 2

In [69]:
### Parameters
namespace = "nci-mimoun-bi-org"
workspace = "CCLF_TSCA"
paths_to_samples_info = "paths_to_batches_info.xlsx"
google_bucket_id = "fc-35446f22-ea37-483a-bd6c-5e9fc56851ff"
batches_info = pd.read_excel(paths_to_samples_info)

#### Deletions

In [ ]:
### Delete unnecessary attributes in samples
attrs = ["annotate_variants_vcf", "annotate_variants_vcf_index", "oncotated_maf", "mutect2_vcf_index", "match_normal_oncotated_maf", "gnomad_vcf", "merged_vcfs", "match_primary_tumor_bam_file"]
for idx, row in all_samples.iterrows():
    fc_interface.delete_entity_attributes(namespace, workspace, "sample", row['entity:sample_id'], attrs)

In [ ]:
### Delete method repostory
firecloud_api.delete_repository_method('tsca', 'CreatePoN', 15)

In [ ]:
### Delete method repository
firecloud_api.delete_repository_method('tsca', 'Mutect2_for_Normal', 1)

In [ ]:
### Delete multiple method repositories
for i in range(10, 23):
    firecloud_api.delete_repository_method('tsca', 'CreatePoN', i)

In [ ]:
### Delete method config
firecloud_api.delete_workspace_config(namespace, workspace, "tsca", "Mutect2_for_Normal")

#### Pair creation

In [ ]:
### Create pairs
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

In [ ]:
### Upload pairs
res = fc_interface.upload_pairs(namespace, workspace, clean_pairs_list)

In [ ]:
# ### Delete unnecessary attributes in pairs
# ### NOT WOKING
# all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
# pairs_list = fc_interface.create_pairs_list(all_samples)
# attrs = ["__annotate_variants_vcf", "__annotate_variants_vcf_index", \
#          "__scattered_intervals", "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2", \
#          "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2"]
# for idx, row in pairs_list.iterrows():
#     fc_interface.delete_entity_attributes(namespace, workspace, "pair", row['entity:pair_id'], attrs)

In [ ]:
### Upload metadata for given batch
fc_interface.update_batch_metadata('TSCA21', \
                                  '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                                      paths_to_samples_info, \
                                        namespace, workspace, google_bucket_id)

In [ ]:
### Delete sample_set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### All paths to batch info
# pd.read_excel('paths_to_batches_info.xlsx').path_to_samples_info.tolist()

In [ ]:
### Create cumulative PoN (all batches)
batches_info = pd.read_excel(paths_to_samples_info)
PoN = fc_interface.create_panel_of_normals(batches_info.path_to_samples_info.tolist(), -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'CumPoN21', namespace, workspace)

In [ ]:
### Create PoN from normals of only one batch (no samples from other batches)
batches_info = pd.read_excel(paths_to_samples_info)
tsca21_path = batches_info.path_to_samples_info.tolist()[-1]
PoN = fc_interface.create_panel_of_normals([tsca21_path], -1, 'PoN_TSCA21_Normals')
fc_interface.upload_pon(PoN, 'PoN_TSCA21_Normals', namespace, workspace)

In [ ]:
### Delete pair set
fc_interface.delete_pair_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Delete sample set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Download method configs
fc_interface.download_method_configs(namespace, workspace)

In [ ]:
### Upload pairsets
tn_pairsets, tp_pairsets = fc_interface.create_pairsets(all_samples, clean_pairs_list)
fc_interface.upload_pairsets(namespace, workspace, tn_pairsets, "TN")
fc_interface.upload_pairsets(namespace, workspace, tp_pairsets, "TP")

In [ ]:
### Delete all pairs
for idx, row in clean_pairs_list.iterrows():
    res = fc_interface.delete_pair(namespace, workspace, row['entity:pair_id'])
    print(res)

In [ ]:
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

### Update pair attributes
update_pair_attrs(namespace, workspace, clean_pairs_list, attrs=['control_sample_tsca_id'])

In [ ]:
### Create PoN from all normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [6]:
### Create PoN with normals from batch + other normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
# PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
# fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [4]:
# batch_id = 'TSCA21'
# batch_path = batches_info.loc[batches_info.tsca_id==batch_id, 'path_to_samples_info'].item()
# pd.read_table(batch_path)
# batch_id, paths, N, name

In [21]:
# Retrieve samples before export to FC
tsca21 = \
    fc_interface.\
        prepare_batch_samples_for_metadata_export('/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                            'TSCA21', google_bucket_id)

In [ ]:
### Update WDLs
fc_interface.download_remote_wdls(namespace, workspace)

---

---

In [93]:
### Upload metadata for given batch
fc_interface.update_batch_metadata('TSCA21', \
                                  '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                                      paths_to_samples_info, \
                                        namespace, workspace, google_bucket_id)

65 Participants in this batch
95 Samples in this batch


In [95]:
(r1, r2, r3, r4, r5, r6) = fc_interface.export_batch_metadata_to_fc('TSCA21', namespace, workspace)

In [97]:
r6.text

'{\n  "statusCode": 400,\n  "source": "rawls",\n  "timestamp": 1506538615260,\n  "causes": [],\n  "stackTrace": [],\n  "message": "Invalid input: Cancer Cell Line Factory (CCLF) / Eli Van Allen/ Toni Choueiri (DFCI) - Kidney. Input may only contain alphanumeric characters, underscores, and dashes."\n}\n'

In [ ]:
"Cancer Cell Line Factory (CCLF) / Eli Van Allen/ Toni Choueiri (DFCI) - Kidney."

In [104]:
import glob

In [105]:
files = glob.glob('walkupseq_files/*')

In [109]:
for f in files[5:]:
    print(f)
    data = pd.read_table(f)
#     print(data.columns)
    print( 'Collection' in data.columns)

walkupseq_files/tsca16_walkupseq_201702_sample_info.txt
False
walkupseq_files/tsca17_walkupseq_201703_sample_info.txt
True
walkupseq_files/tsca18_walkupseq_201704_sample_info.txt
True
walkupseq_files/tsca19_walkupseq_201706_sample_info.txt
True
walkupseq_files/tsca20_walkupseq_201707_sample_info.txt
True
walkupseq_files/tsca21_walkupseq_201708_sample_info.txt
True


In [111]:
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)

In [113]:
all_samples[all_samples.tsca_id=="TSCA15"]

,entity:sample_id,participant_id,clean_bam_file_capture,external_id_validation,aggregation_product_name_validation,bsp_sample_id_validation,stock_sample_id_validation,sample_type,picard_aggregation_type_validation,processed_subtype_validation,source_subtype_validation,squid_sample_id_validation,tumor_subtype,short_letter_code,bam_filename,bai_filename,clean_bai_file_capture,tsca_id
0,AA24-Tumor-SM-DHZ8W,AA24,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA24T_OPAC_p9,TSCA Rapid Cancer Detection Panel v2,SM-DHZ8W,SM-DHTKX,Tumor,PCR,Cells:Pellet frozen,Cells:Growing,AA24T_OPAC_p9,Primary,TP,2_AA24T_OPAC_p9_H5V3YBCXY.2.aligned.duplicates...,2_AA24T_OPAC_p9_H5V3YBCXY.2.aligned.duplicates...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA15
1,AA25-Tumor-SM-DHZ99,AA25,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA25T_OPAC_p4,TSCA Rapid Cancer Detection Panel v2,SM-DHZ99,SM-DHTKY,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA25T_OPAC_p4,Primary,TP,2_AA25T_OPAC_p4_H5V3YBCXY.2.aligned.duplicates...,2_AA25T_OPAC_p4_H5V3YBCXY.2.aligned.duplicates...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA15
2,AA33-Normal-SM-DHZ9L,AA33,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA33N_OPAC_p5,TSCA Rapid Cancer Detection Panel v2,SM-DHZ9L,SM-DHTKZ,Normal,PCR,Cells:Pellet frozen,"Cells:Cell Line, Viable",AA33N_OPAC_p5,NaN,NCELL,2_AA33N_OPAC_p5_H5V3YBCXY.2.aligned.duplicates...,2_AA33N_OPAC_p5_H5V3YBCXY.2.aligned.duplicates...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA15
3,AA33-Tumor-SM-DHZ9X,AA33,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA33T_OPAC_p5,TSCA Rapid Cancer Detection Panel v2,SM-DHZ9X,SM-DHTL1,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA33T_OPAC_p5,Primary,TP,2_AA33T_OPAC_p5_H5V3YBCXY.2.aligned.duplicates...,2_AA33T_OPAC_p5_H5V3YBCXY.2.aligned.duplicates...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA15
4,AA38-Tumor-SM-DHZ98,AA38,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA38T_OPAC_p9,TSCA Rapid Cancer Detection Panel v2,SM-DHZ98,SM-DHTKQ,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA38T_OPAC_p9,Metastatic,TM,2_AA38T_OPAC_p9_H5V3YBCXY.2.aligned.duplicates...,2_AA38T_OPAC_p9_H5V3YBCXY.2.aligned.duplicates...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA15
5,AA39-Tumor-SM-DHZAA,AA39,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA39T_OPAC_p6,TSCA Rapid Cancer Detection Panel v2,SM-DHZAA,SM-DHTL2,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA39T_OPAC_p6,Primary,TP,2_AA39T_OPAC_p6_H5V3YBCXY.2.aligned.duplicates...,2_AA39T_OPAC_p6_H5V3YBCXY.2.aligned.duplicates...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA15
6,AA40-Tumor-SM-DHZ7W,AA40,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA40T_OPAC_p5,TSCA Rapid Cancer Detection Panel v2,SM-DHZ7W,SM-DHTL3,Tumor,PCR,Cells:Pellet frozen,"Cells:Cell Line, Viable",AA40T_OPAC_p5,Primary,TP,2_AA40T_OPAC_p5_H5V3YBCXY.2.aligned.duplicates...,2_AA40T_OPAC_p5_H5V3YBCXY.2.aligned.duplicates...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA15
7,AA42-Tumor-SM-DHZ9W,AA42,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA42T_OPAC_p4,TSCA Rapid Cancer Detection Panel v2,SM-DHZ9W,SM-DHTKS,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA42T_OPAC_p4,Primary,TP,2_AA42T_OPAC_p4_H5V3YBCXY.2.aligned.duplicates...,2_AA42T_OPAC_p4_H5V3YBCXY.2.aligned.duplicates...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA15
8,AA43-Tumor-SM-DHZ8K,AA43,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA43T_OPAC_p7,TSCA Rapid Cancer Detection Panel v2,SM-DHZ8K,SM-DHTKW,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA43T_OPAC_p7,Metastatic,TM,2_AA43T_OPAC_p7_H5V3YBCXY.2.aligned.duplicates...,2_AA43T_OPAC_p7_H5V3YBCXY.2.aligned.duplicates...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA15
9,AA44-Normal-SM-DHZ88,AA44,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA44N_OPAC_p7,TSCA Rapid Cancer Detection Panel v2,SM-DHZ88,SM-DHTKV,Normal,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA